In [ ]:
import sys
import math
import random
from itertools import permutations
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import csv

In [ ]:
# number of locations, including the depot. The index of the depot is 0
n = 10
locations = [*range(n)]

# number of vans
K = 3
vans = [*range(K)]

In [ ]:
#Tabela das origens de destinos
path = '/Users/TullioPires/Python-Mestrado/Problema Celio/celio-problema.csv'
celio = pd.read_csv(path, index_col=[0,1], squeeze=True)

#Pivotar a tabela, para ler
pivoted_celio = celio.reset_index().pivot(index = 'Origem', columns = 'Destino', values = 'Distancia')
pivoted_celio

In [ ]:
time = {}
with open(path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        i = int(row['Origem'])
        j = int(row['Destino'])
        distance = int(row['Distancia'])
        time[(i, j)] = distance
        
#Mostrando as distancias como dicionário
for (i, j), distance in time.items():
    print(f"Distance from {i} to {j}: {distance}")

In [ ]:
#Modelo Celio Gurobi 2
celio_gurobi = gp.Model('modelo_celio_gurobi.lp')

In [ ]:
#criando variáveis
# x=1 se a van k visita e vai direto do ponto i pro j
x = celio_gurobi.addVars(time.keys(), vans, vtype=GRB.BINARY, name='FromToBy') 

In [ ]:
#criando variáveis
# y=1 se o cliente i é visitado pela van k
y = celio_gurobi.addVars(locations, vans, vtype=GRB.BINARY, name='visitBy')

In [ ]:
#criando variáveis
# O Nº de vans usadas é uma variável de decisão
z = celio_gurobi.addVars(vans, vtype=GRB.BINARY, name='used')

In [ ]:
#criando variáveis
# tempo de viagem por van
t = celio_gurobi.addVars(vans, ub=240, name='travelTime')

In [ ]:
#criando variáveis
#Tempo de viagem máximo
s = celio_gurobi.addVar(name='maxTravelTime')

In [ ]:
#Criando restrições
# Restrição de utilização das vans

visitCustomer = celio_gurobi.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )

In [ ]:
# Restrição de tempo
# Excluindo o tempo de retorno para a base

travelTime = celio_gurobi.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in vans), 
                          name='travelTimeConstr' )

In [ ]:
# Restrição de visita a TODOS os clientes
visitAll = celio_gurobi.addConstrs((y.sum(i,'*') == 1 for i in locations if i > 0), name='visitAll' )

In [ ]:
#Restrição do depósito, de que as vans voltam
depotConstr = celio_gurobi.addConstrs((y[0, k] == z[k] for k in vans), name='depotConstr' )

In [ ]:
# Restrição de chegada nos clientes
#Se a localidade j é visitada pela van k, então ela vem de i
ArriveConstr = celio_gurobi.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

In [ ]:
#Restrição de saída do cliente
#Se a van k sai de j, então ela vai pra i
LeaveConstr = celio_gurobi.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

In [ ]:
#restrição de quebra de simetria
breakSymm = celio_gurobi.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in vans if k>0), name='breakSymm' )

In [ ]:
#Restrição de tempo de viagem máximo
maxTravelTime = celio_gurobi.addConstrs((t[k] <= s for k in vans), name='maxTravelTimeConstr')

In [ ]:
#Funções objetivos
celio_gurobi.ModelSense = GRB.MINIMIZE
celio_gurobi.setObjectiveN(z.sum(), 0, priority=1, name="Number of vans")
celio_gurobi.setObjectiveN(s, 1, priority=0, name="Travel time")

In [ ]:
#Função callback
#Criamos para evitar que existam rotas nas quais não se comece ou não se termine na base

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        #Faça uma lista das arestas selecionadas na solução
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        #Ache o menor ciclo na lista de arestas selecionadas
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


#Dada uma tuplelist de quinas, ache a menor subtour que não contenha a base (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # Tamanho inicial tem 1 cidade a mais
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [ ]:
# Verificando a formulação do modelo

celio_gurobi.write('modelo_celio_gurobi.lp')

In [ ]:
# Rodando a otimização
celio_gurobi._x = x
celio_gurobi.Params.LazyConstraints = 1
celio_gurobi.optimize(subtourelim)

In [ ]:
# Print optimal routes
for k in vans:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for van {k}: {i}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            print(f" -> {i}", end='')
            if i == 0:
                break
        print(f". Travel time: {round(t[k].X,2)} min")

print(f"Max travel time: {round(s.X,2)}")